In [1]:
import pandas as pd;

In [2]:
#CONFIGURATION ----------------------------------------------------------------------------------------------------------

CSV_SUB_DIR="entities" #the same defined in Notebook 1
DATABASE_URI="bolt://localhost:7687";
DB_NAME="migrateddatabase"; #WARNING! After the selection of a database, all its contents will be deleted!

USER="neo4j";
PASSWORD="root";

#------------------------------------------------------------------------------------------------------------------------

# Notebook 2: Import CSV in Neo4j

Second part of the migration procedure. It consists in using the previously created CSV files for setting up the Neo4j DB.

In [3]:
#Neo4j connector class

from neo4j import GraphDatabase
# From https://neo4j.com/developer/python/

class NeoConnector:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=DB_NAME):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response
    def load_node_web_site_csv(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/node_web_site.csv" AS row
                MERGE (w:WEB_SITE {name:row.url_id, domain_name_id:row.domain_name_id})
                RETURN count(w)'''
        return self.query(query);
    def load_node_landing_name_csv(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/node_landing_name.csv" AS row
                MERGE (w:WEB_SERVER {name:row.domain_name, landing_url:row.landing_url, landing_https:row.landing_https})
                RETURN count(w)''';
        return self.query(query);
    def load_node_network_csv(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/node_network.csv" AS row
                MERGE (w:NETWORK {name:row.ip_network_id,roa:row.state})
                RETURN count(w)''';
        return self.query(query);
    def load_node_name_server_csv(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/node_name_server.csv" AS row
                MERGE (ns:NAME_SERVER {name:row.name_id})
                RETURN count(ns)''';
        return self.query(query);
    def load_node_zone_csv(self): 
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/node_zone.csv" AS row
                CREATE (ns:ZONE {name:row.name})
                RETURN count(ns)''';
        return self.query(query);   
    def load_node_zone_csv_old(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/node_zone.csv" AS row
                MERGE (ns:ZONE {name:row.name})
                RETURN count(ns)''';
        return self.query(query);   
    def load_node_aut_sys_csv(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/node_aut_sys.csv" AS row
                MERGE (as:AUTONOMOUS_SYSTEM {name:row.number,description:row.description,country_code:row.country_code})
                RETURN count(as)''';
        return self.query(query); 
    def load_node_mail_domain_csv(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/node_mail_domain.csv" AS row
                MERGE (md:MAIL_DOMAIN {name:row.name_id})
                RETURN count(md)''';
        return self.query(query);
    def load_node_mail_server_csv(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/node_mail_server.csv" AS row
                MERGE (ms:MAIL_SERVER {name:row.name_id})
                RETURN count(ms)''';
        return self.query(query);                
    def load_rel_land_csv(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_land.csv" AS row
                MATCH (w:WEB_SITE {name:row.url_id})
                MATCH (l:WEB_SERVER {landing_url:row.landing_url})
                MERGE (w)-[:MAPPED_IN {from_https:row.starting_https, to_https:row.landing_https,landing_url:l.landing_url}]->(l)
                RETURN count(w)''';
        return self.query(query);
    def load_rel_cname_ln_csv(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_cname_ln.csv" AS row
                MATCH (l1:WEB_SERVER {name:row.name_1})
                MATCH (l2:WEB_SERVER {name:row.name_2})
                MERGE (l1)-[:CNAME]->(l2)
                RETURN count(l1)''';
        return self.query(query);
    def load_rel_located_ln_csv(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_located_ln.csv" AS row
                MATCH (l:WEB_SERVER {name:row.name_id})
                MATCH (n:NETWORK {name:row.ip_network_id})
                MERGE (l)-[:LOCATED {ip_address:row.ip_address_id}]->(n)
                RETURN count(l)''';
        return self.query(query);
    def load_rel_belong_ns_csv(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_belong_ns.csv" AS row
                CREATE (ns:NAME_SERVER {name:row.ns_name})-[:BELONG]->(z:ZONE {name:row.ns_zone})
                RETURN count(ns)''';
        return self.query(query);
    def load_rel_belong_ns_csv_old(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_belong_ns.csv" AS row
                MATCH (ns:NAME_SERVER {name:row.ns_name})
                MATCH (z:ZONE {name:row.ns_zone})
                MERGE (ns)-[:BELONG]->(z)
                RETURN count(ns)''';
        return self.query(query);

    def load_rel_belong_ms_csv(self): 
        query='''
        LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_belong_ms.csv" AS row
        CREATE (ms:MAIL_SERVER {name:row.ms_name})-[:BELONG]->(z:ZONE {name:row.ms_zone})
        RETURN count(ms)''';
        return self.query(query);

    def load_rel_belong_ms_csv_old(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_belong_ms.csv" AS row
                MATCH (ms:MAIL_SERVER {name:row.ms_name})
                MATCH (z:ZONE {name:row.ms_zone})
                MERGE (ms)-[:BELONG]->(z)
                RETURN count(ms)''';
        return self.query(query);

    def load_rel_belong_md_csv(self): 
        query='''
        LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_belong_md.csv" AS row
        CREATE (md:MAIL_DOMAIN {name:row.md_name})-[:BELONG]->(z:ZONE {name:row.md_zone})
        RETURN count(md)''';
        return self.query(query);

    def load_rel_belong_md_csv_old(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_belong_md.csv" AS row
                MATCH (md:MAIL_DOMAIN {name:row.md_name})
                MATCH (z:ZONE {name:row.md_zone})
                MERGE (md)-[:BELONG]->(z)
                RETURN count(md)''';
        return self.query(query);
    def load_rel_belong_ln_csv(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_belong_ln.csv" AS row
                MATCH (ln:WEB_SERVER {name:row.ln_name})
                MATCH (z:ZONE {name:row.ln_zone})
                MERGE (ln)-[:BELONG]->(z)
                RETURN count(ln)''';
        return self.query(query);
    def load_rel_located_ns_csv(self):
        query='''
        LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_located_ns.csv" AS row
        CREATE (ns:NAME_SERVER {name:row.name_id})-[:LOCATED{ip_address:row.ip_address_id}]->(n:NETWORK {name:row.ip_network_id})
        RETURN count(ns)''';
        return self.query(query);
    def load_rel_located_ns_csv_old(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_located_ns.csv" AS row
                MATCH (ns:NAME_SERVER {name:row.name_id})
                MATCH (n:NETWORK {name:row.ip_network_id})
                MERGE (ns)-[:LOCATED{ip_address:row.ip_address_id}]->(n)
                RETURN count(ns)''';
        return self.query(query);

    def load_rel_located_ms_csv(self):
        query='''
            LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_located_ms.csv" AS row
            CREATE (ms:MAIL_SERVER {name:row.name_id})-[:LOCATED{ip_address:row.ip_address_id}]->(n:NETWORK {name:row.ip_network_id})
            RETURN count(ms)''';
        return self.query(query);

    def load_rel_located_ms_csv_old(self): 
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_located_ms.csv" AS row
                MATCH (ns:MAIL_SERVER {name:row.name_id})
                MATCH (n:NETWORK {name:row.ip_network_id})
                MERGE (ns)-[:LOCATED{ip_address:row.ip_address_id}]->(n)
                RETURN count(ns)''';
        return self.query(query);
    def load_rel_cname_ns_csv(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_cname_ns.csv" AS row
                MATCH (ns1:NAME_SERVER {name:row.name_1})
                MATCH (ns2:NAME_SERVER {name:row.name_2})
                MERGE (ns1)-[:CNAME]->(ns2)
                RETURN count(ns1)''';
        return self.query(query);
    def load_rel_cname_ms_csv(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_cname_ms.csv" AS row
                MATCH (ns1:MAIL_SERVER {name:row.name_1})
                MATCH (ns2:MAIL_SERVER {name:row.name_2})
                MERGE (ns1)-[:CNAME]->(ns2)
                RETURN count(ns1)''';
        return self.query(query);
    def load_rel_managed_csv(self):  
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_managed.csv" AS row
                CREATE (n:NETWORK {name:row.ip_network_id})-[:MANAGED_BY]->(as:AUTONOMOUS_SYSTEM {name:row.number})
                RETURN count(n)''';
        return self.query(query);


    def load_rel_managed_csv_old(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_managed.csv" AS row
                MATCH (n:NETWORK {name:row.ip_network_id})
                MATCH (as:AUTONOMOUS_SYSTEM {name:row.number})
                MERGE (n)-[:MANAGED_BY]->(as)
                RETURN count(n)''';
        return self.query(query);

    def load_rel_composed_by_csv(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_composed_by.csv" AS row
                CREATE (z:ZONE {name:row.zone_name})-[:COMPOSED_BY]->(ns:NAME_SERVER {name:row.ns_name})
                RETURN count(z)''';
        return self.query(query);
    def load_rel_depend_csv(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_depend.csv" AS row
                CREATE (z1:ZONE {name:row.name})-[:DEPEND]->(z2:ZONE {name:row.dependency_id})
                RETURN count(z1)''';
        return self.query(query);
    def load_rel_parent_csv(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_parent.csv" AS row
                CREATE (z1:ZONE {name:row.name})-[:PARENT]->(z2:ZONE {name:row.dependency_id})
                RETURN count(z1)''';
        return self.query(query);
    def load_rel_mapped_in_csv(self): 
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_mapped_in.csv" AS row
                CREATE (md:MAIL_DOMAIN {name:row.md_name})-[:MAPPED_IN]->(ms:MAIL_SERVER {name:row.ms_name})
                RETURN count(md)''';
        return self.query(query);

    def load_rel_mapped_in_csv_old(self):
        query='''
                LOAD CSV WITH HEADERS FROM "file:///'''+CSV_SUB_DIR+'''/rel_mapped_in.csv" AS row
                MATCH (md:MAIL_DOMAIN {name:row.md_name})
                MATCH (ms:MAIL_SERVER {name:row.ms_name})
                MERGE (md)-[:MAPPED_IN]->(ms)
                RETURN count(md)''';
        return self.query(query);

In [4]:
connector=NeoConnector(uri=DATABASE_URI,user=USER,pwd=PASSWORD);
connector.query("MATCH (n) DETACH DELETE n"); #Delete DB content

Export entities. Functions are called in different blocks to provide an effective indication of which functions are slower then the others:

In [5]:
#NODES: WEBSITE, LANDING_NAME, NETWORK
connector.load_node_web_site_csv();
connector.load_node_landing_name_csv();
connector.load_node_network_csv();

In [6]:
#NODES: ZONE, NAME_SERVER, AUTONOMOUS_SYSTEM
connector.load_node_zone_csv();
connector.load_node_name_server_csv();
connector.load_node_aut_sys_csv();

In [7]:
#NODES: MAIL_DOMAIN, MAIL_SERVER
connector.load_node_mail_domain_csv();
connector.load_node_mail_server_csv();

In [8]:
#RELATIONSHIPS: LAND, CNAME, LOCATED
connector.load_rel_land_csv();
connector.load_rel_cname_ln_csv();
connector.load_rel_located_ln_csv();

In [9]:
#RELATIONSHIPS: BELONG LN -> ZONE
connector.load_rel_belong_ln_csv();

In [10]:
#RELATIONSHIPS: BELONG NS -> ZONE
connector.load_rel_belong_ns_csv();

In [11]:
#RELATIONSHIPS: LOCATED NS -> NETWORK
connector.load_rel_located_ns_csv();

In [12]:
#RELATIONSHIPS: CNAME NS -> NS
connector.load_rel_cname_ns_csv();

In [13]:
#RELATIONSHIPS: COMPOSED_BY, DEPEND, PARENT
connector.load_rel_composed_by_csv();
connector.load_rel_parent_csv();

#connector.load_rel_depend_csv(); #DEPEND relationship is not imported anymore.


In [14]:

#RELATIONSHIPS: BELONG MS -> ZONE
connector.load_rel_belong_ms_csv();

In [15]:
#RELATIONSHIPS: BELONG MD -> ZONE
connector.load_rel_belong_md_csv();

In [16]:
#RELATIONSHIPS: LOCATED MS -> NETWORK
connector.load_rel_located_ms_csv();

In [17]:
#RELATIONSHIPS: MANAGED_BY
connector.load_rel_managed_csv();

In [18]:
#RELATIONSHIPS: CNAME MS -> MS
connector.load_rel_cname_ms_csv();

In [19]:
#RELATIONSHIPS: MAPPED_IN
connector.load_rel_mapped_in_csv();

In order to speed up the queries above, some duplicates have been created. Let's refactor graph by eliminating node duplicates:

In [20]:
connector.query("MATCH (n:NAME_SERVER) WITH n.name AS name, COLLECT(n) AS nodelist, COUNT(*) AS count WHERE count>1 CALL apoc.refactor.mergeNodes(nodelist) YIELD node RETURN node");
connector.query("MATCH (n:ZONE) WITH n.name AS name, COLLECT(n) AS nodelist, COUNT(*) AS count WHERE count > 1 CALL apoc.refactor.mergeNodes(nodelist) YIELD node RETURN node");
connector.query("MATCH (n:NETWORK) WITH n.name AS name, COLLECT(n) AS nodelist, COUNT(*) AS count WHERE count > 1 CALL apoc.refactor.mergeNodes(nodelist) YIELD node RETURN node");
connector.query("MATCH (n:AUTONOMOUS_SYSTEM) WITH n.name AS name, COLLECT(n) AS nodelist, COUNT(*) AS count WHERE count > 1 CALL apoc.refactor.mergeNodes(nodelist) YIELD node RETURN node");
connector.query("MATCH (n:MAIL_DOMAIN) WITH n.name AS name, COLLECT(n) AS nodelist, COUNT(*) AS count WHERE count > 1 CALL apoc.refactor.mergeNodes(nodelist) YIELD node RETURN node");
connector.query("MATCH (n:MAIL_SERVER) WITH n.name AS name, COLLECT(n) AS nodelist, COUNT(*) AS count WHERE count > 1 CALL apoc.refactor.mergeNodes(nodelist) YIELD node RETURN node");
connector.query("MATCH (n:NETWORK)-[r:MANAGED_BY]->(q:AUTONOMOUS_SYSTEM) WITH n.name AS name, COLLECT(r) AS relist, COUNT(*) AS count WHERE count > 1 CALL apoc.refactor.mergeRelationships(relist) YIELD rel RETURN rel");
connector.query("MATCH (n:WEB_SERVER) WITH n.name AS name, COLLECT(n) AS nodelist, COUNT(*) AS count WHERE count > 1 CALL apoc.refactor.mergeNodes(nodelist) YIELD node RETURN node");
connector.query("MATCH (n:ZONE) WHERE NOT EXISTS(n.name) DETACH DELETE n")

[]

21/06 - Modification required for the "LANDING NAME to WEB_SERVER" change.

In [21]:
connector.query("MATCH (ws:WEB_SERVER) REMOVE ws.landing_https REMOVE ws.landing_url RETURN count(ws)");

[<Record count(ws)=0>]